# Pre-processing Lake Temperature

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm

tqdm.pandas()

In [6]:
data_path = "../data/raw/lake_temperature"
train_path = "all_lakes_inputs"
train_file = "all_lakes_historical_training.csv"
meteo_path = "68_lakes_meteo"
ice_path = "68_pretrainer_ice_flags"
test_path = "all_lakes_evaluation"

In [3]:
"nhd_1097324_meteo.csv"[:-10]

'nhd_1097324'

In [11]:
def get_site_id(filename):
    # site id = filename without "_meteo.csv"
    site_id = filename[:-10]
    
    return site_id

In [24]:
df = pd.read_csv(os.path.join(data_path, train_path, train_file))
df

,site_id,date,depth,temp
0,nhd_10596466,2010-07-21,0.00,26.94
1,nhd_10596466,2010-07-21,1.00,26.73
2,nhd_10596466,2010-07-21,2.00,26.60
3,nhd_10596466,2010-07-21,3.00,26.42
4,nhd_10596466,2010-07-21,4.00,24.32
...,...,...,...,...
252578,nhd_9022741,2018-06-11,12.19,7.22
252579,nhd_9022741,2018-06-11,13.72,6.72
252580,nhd_9022741,2018-06-11,15.24,5.89
252581,nhd_9022741,2018-06-11,18.29,5.50


In [29]:
def create_training_set(data_path, train_path, train_file, meteo_path, ice_path):
    path = os.path.join(data_path, train_path)
    
    # Get all site ids
    site_ids = [get_site_id(f) for f in os.listdir(os.path.join(path, meteo_path)) if f.endswith(".csv")]
  
    # For each site id:
    train_file = os.path.join(data_path, train_path, train_file)
    train_df = pd.read_csv(train_file)
    
    meteo_df = []
    ice_df = []
    
    for site_id in tqdm(site_ids, desc="Creating input dataframe"):
        # Read data and append to list
        meteo_file = os.path.join(path, meteo_path, site_id+"_meteo.csv")
        ice_file = os.path.join(path, ice_path, site_id+"_ice_flag.csv")
                
        meteo = pd.read_csv(meteo_file, index_col=None, header=0, sep=",")
        meteo["site_id"] = site_id
        meteo_df.append(meteo)
        
        ice = pd.read_csv(ice_file, index_col=None, header=0, sep=",")
        ice["site_id"] = site_id
        ice_df.append(ice)
                
    # Convert list of dataframes to dataframe
    meteo_df = pd.concat(meteo_df, axis=0, ignore_index=True).set_index(["date", "site_id"])
    ice_df = pd.concat(ice_df, axis=0, ignore_index=True).set_index(["date", "site_id"])
    
    train_df = train_df.join(meteo_df, on=["date", "site_id"])
    train_df = train_df.join(ice_df, on=["date", "site_id"])
    
    return train_df

In [31]:
df = create_training_set(data_path, train_path, train_file, meteo_path, ice_path)

['nhd_120052233', 'nhd_2723765', 'nhd_4250586', 'nhd_10596466', 'nhd_14783883', 'nhd_13393533', 'nhd_2384766', 'nhd_1102088', 'nhd_1101636', 'nhd_1101812', 'nhd_1101590', 'nhd_4250588', 'nhd_13343906', 'nhd_1097324', 'nhd_1099420', 'nhd_13294464', 'nhd_120052351', 'nhd_1102086', 'nhd_2353946', 'nhd_13293262', 'nhd_1099526', 'nhd_2385444', 'nhd_1101938', 'nhd_2648034', 'nhd_13344056', 'nhd_13344210', 'nhd_1099282', 'nhd_1099430', 'nhd_120052022', 'nhd_2384584', 'nhd_1099240', 'nhd_1101506', 'nhd_1101544', 'nhd_1101504', 'nhd_1100962', 'nhd_1101754', 'nhd_1101942', 'nhd_1099432', 'nhd_1101864', 'nhd_120051922', 'nhd_1102366', 'nhd_13344284', 'nhd_2385496', 'nhd_120053694', 'nhd_2385648', 'nhd_120052587', 'nhd_13631637', 'nhd_1099052', 'nhd_12160852', 'nhd_2723803', 'nhd_9022741', 'nhd_2647582', 'nhd_2349198', 'nhd_13393567', 'nhd_13290898', 'nhd_1099136', 'nhd_1099172', 'nhd_13344170', 'nhd_2349188', 'nhd_1101616', 'nhd_1102284', 'nhd_1099476', 'nhd_2723871', 'nhd_1101684', 'nhd_12162854

Creating input dataframe:   0%|          | 0/68 [00:00<?, ?it/s]

In [32]:
df

,site_id,date,depth,temp,ShortWave,LongWave,AirTemp,RelHum,WindSpeed,Rain,Snow,ice
0,nhd_10596466,2010-07-21,0.00,26.94,299.822167,353.923335,23.741252,61.952459,3.412611,0.007915,0.0,False
1,nhd_10596466,2010-07-21,1.00,26.73,299.822167,353.923335,23.741252,61.952459,3.412611,0.007915,0.0,False
2,nhd_10596466,2010-07-21,2.00,26.60,299.822167,353.923335,23.741252,61.952459,3.412611,0.007915,0.0,False
3,nhd_10596466,2010-07-21,3.00,26.42,299.822167,353.923335,23.741252,61.952459,3.412611,0.007915,0.0,False
4,nhd_10596466,2010-07-21,4.00,24.32,299.822167,353.923335,23.741252,61.952459,3.412611,0.007915,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
252578,nhd_9022741,2018-06-11,12.19,7.22,314.067376,312.102917,16.682920,70.277152,5.742651,0.000000,0.0,False
252579,nhd_9022741,2018-06-11,13.72,6.72,314.067376,312.102917,16.682920,70.277152,5.742651,0.000000,0.0,False
252580,nhd_9022741,2018-06-11,15.24,5.89,314.067376,312.102917,16.682920,70.277152,5.742651,0.000000,0.0,False
252581,nhd_9022741,2018-06-11,18.29,5.50,314.067376,312.102917,16.682920,70.277152,5.742651,0.000000,0.0,False
